# ACID Properties

### Atomicity
Transactions might be incomplete due to some reasons;
 - Transaction may be aborted
 - Transaction may be terminated unsuccesfully
 - System may crash
 - Transaction may terminate itself because of some unexpected situations

Transactions should not be partially successful. One of the reasons is shown below

Imagine a transaction of money from userA's account to userB's
#### ATM
|userA|userB| 
|-----|-----|
|5000 |2000 |

userA sends 1000 units amount of money to userB but the system crashes after the money is sent but not deposited to userB's account

#### ATM
|userA|userB| 
|-----|-----|
|4000 |2000 |

userA lost 1000 units of money.
This needs to be avoided so a system that does all transactions or none is needed(**atomicity**).

Now lets restart the process where userA has 5000 and userB has 2000.
If the system crashes again this time the table would look like

#### ATM
|userA|userB| 
|-----|-----|
|5000 |2000 |

Because partial transaction is not allowed and the system **undid the actions of partial transaction**.

### Durability
When rolling back to the initial state a structure called **log** is being used. This structure keeps the changes and when the program asks for rollback, the log undo the changes. The record of the changes, log, ensures the ***durability*** of the database.

#### ATM
|userA|userB| 
|-----|-----|
|5000 |2000 |

Transaction from userA to userB. The amount is 1000 units.

#### log
|log  |
|-----|
|userA = 5000|
|userB = 2000|

Push the state at the moment(before transasction) to log.

#### ATM
|userA|userB| 
|-----|-----|
|4000 |3000 |

If any crush has been occured, then it will rollback (the values in log would be called).


The example below provides some insight of atomicity(with all or nothing property) and durability(with stack based log).

In [2]:
class ATM:
    def __init__(self):
        self.userA = 5000
        self.userB = 2000
        self.log = []  # Stack-based log to store transaction changes
        
    def rollback_all(self):
        while self.log:
            self.userA, self.userB = self.log.pop()  # Revert to the initial state from the log

    def transfer(self, amount):
        try:
            # Simulating the transaction
            self.log.append((self.userA, self.userB))  # Log the initial state
            self.userA -= amount
            self.userB += amount

            # Simulate system crash
            raise Exception("Simulated system crash after transaction")

        except Exception as e:
            print("Transaction aborted due to system crash:", e)
            # Rollback changes due to crash
            atm.rollback_all()  # Revert to the initial state from the log

    def safe_transfer(self, amount):
        try:
            # Simulating the transaction
            self.log.append((self.userA, self.userB))  # Log the initial state
            self.userA -= amount
            self.userB += amount
            print(f"Transaction successful! userA: {self.userA}, userB: {self.userB}")

        except Exception as e:
            print("Transaction failed:", e)

    def unsafe_transfer(self, amount):
        try:
            # Simulating transaction without handling errors
            self.log.append((self.userA, self.userB))  # Log the initial state
            self.userA -= amount
            raise Exception("Simulated error during transaction")
            self.userB += amount  # This line won't be executed if the exception occurs

        except Exception as e:
            print("Unsafe transaction failed:", e)
            print(f"Transaction without rolling back changes! userA: {self.userA}, userB: {self.userB}")


# Initialize ATM
atm = ATM()

print("Initial state:")
print("userA:", atm.userA, "userB:", atm.userB)

print("\nSimulating a safe transaction (with system crash)")
atm.transfer(1000)

print("\nState after transaction attempt (due to crash):")
print("userA:", atm.userA, "userB:", atm.userB)

print("\nSimulating a safe transaction (without system crash)")
atm.safe_transfer(2000)

print("\nSimulating an unsafe transaction")
atm.unsafe_transfer(500)

Initial state:
userA: 5000 userB: 2000

Simulating a safe transaction (with system crash)
Transaction aborted due to system crash: Simulated system crash after transaction

State after transaction attempt (due to crash):
userA: 5000 userB: 2000

Simulating a safe transaction (without system crash)
Transaction successful! userA: 3000, userB: 4000

Simulating an unsafe transaction
Unsafe transaction failed: Simulated error during transaction
Transaction without rolling back changes! userA: 2500, userB: 4000


### Consistency
Consistentcy refers to the correctness and the integrity of data before and after transaction.

#### ATM
|userA|userB| 
|-----|-----|
|5000 |2000 |

Sum of userA and userB is 7000. The sum shouldn't change when there is a transaction from one to another.
Transaction from userA to userB. The amount is 1000 units.

#### ATM
|userA|userB| 
|-----|-----|
|4000 |3000 |



In [7]:
class ATM:
    def __init__(self):
        self.accounts = {
            'userA': 5000,
            'userB': 2000
        }

    def transfer_money(self, sender, receiver, amount):
        sender_balance = self.accounts.get(sender)
        receiver_balance = self.accounts.get(receiver)

        try:
            if sender_balance is None or receiver_balance is None: # Check if the data is correct
                raise ValueError("Invalid sender or receiver")

            if sender_balance < amount:                            # Check if the data is correct
                raise ValueError("Insufficient funds")

            self.accounts[sender] -= amount
            self.accounts[receiver] += amount

            print(f"\nTransaction successful! {sender} balance: {self.accounts[sender]}, {receiver} balance: {self.accounts[receiver]}")

        except ValueError as e:
            print(f"Transaction failed: {e}. Rolling back changes.")
            if sender_balance is not None:
                self.accounts[sender] = sender_balance
            if receiver_balance is not None:
                self.accounts[receiver] = receiver_balance

    def is_consistent(self):
        userA_before = self.accounts.get('userA', 0)
        userB_before = self.accounts.get('userB', 0)
        before_transaction_sum = userA_before + userB_before

        self.transfer_money('userA', 'userB', 100)

        userA_after = self.accounts.get('userA', 0)
        userB_after = self.accounts.get('userB', 0)
        after_transaction_sum = userA_after + userB_after

        print("\nSum of userA and userB before transaction:", userA_before, "+", userB_before, "=", before_transaction_sum)
        print("Sum of userA and userB after transaction:", userA_after, "+", userB_after, "=", after_transaction_sum)

        return before_transaction_sum == after_transaction_sum

    def display_accounts(self):
        print("Account balances->", " ".join([f"{user}: {balance}" for user, balance in self.accounts.items()]))


atm = ATM()

atm.display_accounts()

print("\nIs the system consistent after a transaction? ", atm.is_consistent())

Account balances-> userA: 5000 userB: 2000

Transaction successful! userA balance: 4900, userB balance: 2100

Sum of userA and userB before transaction: 5000 + 2000 = 7000
Sum of userA and userB after transaction: 4900 + 2100 = 7000

Is the system consistent after a transaction?  True


### Isolation
Even though the transactions may be executed concurrently, there must be just one transaction that is accessing the data in database.

Imagine two people are trying to purchase the last seat in cinema online. Only one can get it right, so the system must allow the entrance of one to the database and then the other. It cannot let both enter the database at the same time because then the updated version cannot be seen to any and they will both be able to purchase the ticket.  The second person couldn't be able to purchase because the other person got into the database and updated it first. Second person would encounter an error like that seat is already purchased.

In [8]:
class CinemaBooking:
    def __init__(self):
        self.available_seats = 1  # Only one seat available initially

    def purchase_seat(self, person):
        try:
            if self.available_seats == 0:
                raise ValueError("Seat is already purchased")

            self.available_seats -= 1
            print(f"{person} purchased the seat successfully!")

        except ValueError as e:
            print(f"{person} encountered an error: {e}")


# Simulating two people trying to purchase the last seat concurrently
def simulate_concurrent_purchases():
    cinema = CinemaBooking()

    # Person 1 attempts to purchase the seat
    cinema.purchase_seat("Person 1")

    # Person 2 attempts to purchase the seat
    cinema.purchase_seat("Person 2")


simulate_concurrent_purchases()


Person 1 purchased the seat successfully!
Person 2 encountered an error: Seat is already purchased


### Author
Orkun Kınay